In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("GOOGLE_API_KEY")


In [3]:
# gemini live api with async and custom function call dynamically

import asyncio 
from google import genai
from google.genai import types
import nest_asyncio
nest_asyncio.apply()

client = genai.Client()
model = "gemini-live-2.5-flash-preview"


def open_shop():
    return "opened"

def close_shop():
    return "closed"




shop_open = types.FunctionDeclaration(name="open_shop",behavior="NON_BLOCKING")
shop_close = types.FunctionDeclaration(name="close_shop")


tools = [types.Tool(function_declarations=[shop_open, shop_close])]



config = types.LiveConnectConfig(
     response_modalities = ["TEXT"],
     tools=tools
)


async def main():
    async with client.aio.live.connect(model=model, config = config) as session:
        prompt = "open the shop and tell me what happened"
        await session.send_client_content(turns ={"parts":[{"text":prompt}]})

        async for chunk in session.receive():
            #print(chunk)
            if chunk.server_content:
                if chunk.text is not None:
                    print(chunk.text,end="")
            if chunk.tool_call:
                function_responses = []
                for fc in chunk.tool_call.function_calls:
                    #print(fc)
                    result = open_shop() if fc.name == "open_shop" else close_shop()
                    function_response = types.FunctionResponse(
                        id = fc.id,
                        name = fc.name,
                        response = {"result":result,"scheduling":"WHEN_IDLE"}
                    )
                    function_responses.append(function_response)
                await session.send_tool_response(function_responses=function_responses)
                #await session.send_tool_response(function_responses)

await main()

The shop is now open.

In [4]:
# gemini live api with Code execution enabled 

import asyncio
from google import genai
from google.genai import types



client = genai.Client()

# for live, we should use this model only
model = "gemini-live-2.5-flash-preview"


tools = [{'code_execution':{}}]
config = {"response_modalities":["Text"],"tools":tools}

async def main():
    async with client.aio.live.connect(model=model,config=config) as session:
        prompt = "Compute the code for face detection using open cv"
        await session.send_client_content(turns={"parts":[{"text":prompt}]})

        async for chunk in session.receive():
            if chunk.server_content:
                if chunk.text is not None:
                    print("A ",chunk.text)
                model_turn = chunk.server_content.model_turn
                if model_turn :
                    for part in model_turn.parts:
                        if part.executable_code is not None:
                            print("B ",part.executable_code.code)
                        if part.code_execution_result is not None:
                            print("C ",part.code_execution_result.output)



await main()

A  To
A   detect faces using OpenCV, you typically follow these steps:

1.  **
A  Load the pre-trained Haar Cascade classifier:** OpenCV provides pre-trained Haar Cascades for various object detections, including frontal faces.
2.  **Load the image:** Read the image on which you want to perform face detection.
3.  
A  **Convert to grayscale:** Haar Cascades work best on grayscale images, as color information is not crucial for feature detection in this method.
4.  **Perform face detection:** Use the loaded classifier to detect faces in the grayscale image. This function returns
A   a list of rectangles, where each rectangle represents a detected face.
5.  **Draw bounding boxes:** Iterate through the detected faces and draw a rectangle (bounding box) around each face on the original image.
6.  **Display
A   the image:** Show the image with the detected faces.

Here's the Python code to achieve this:




B  import cv2
import matplotlib.pyplot as plt

# 1. Load the pre-trained Haar Cascade classifier for face detection
# You need to have the 'haarcascade_frontalface_default.xml' file
# in the same directory as your script, or provide the full path to it.
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Check if the cascade classifier loaded successfully
if face_cascade.empty():
    print("Error: Could not load face cascade classifier.")
else:
    print("Face cascade classifier loaded successfully.")

    # Create a dummy image for demonstration
    # In a real scenario, you would load an image from a file or camera.
    # For now, let's create a black image and pretend it has a face.
    # A better approach for demonstration is to use an image URL or a sample image.
    # Let's use a sample image that we can download.
    # For a real application, you would replace this with your image loading logic.

    # If you have an image file,

B  import cv2
import matplotlib.pyplot as plt
import numpy as np # Import numpy

# 1. Load the pre-trained Haar Cascade classifier for face detection
# Try accessing the cascade from cv2.data, which is the standard way.
# If it fails, we will assume the file is in the current directory.
cascade_path = None
try:
    cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(cascade_path)
    if face_cascade.empty():
        print(f"Error: Could not load face cascade classifier from {cascade_path}.")
        # Fallback if loading from the standard path fails
        print("Attempting to load from current directory: haarcascade_frontalface_default.xml")
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
except AttributeError:
    print("cv2.data.haarcascades attribute not found. Attempting to load from current directory: haarcascade_frontalface_default.xml")
    face_cascade = cv2.CascadeClassifier

C  cv2.data.haarcascades attribute not found. Attempting to load from current directory: haarcascade_frontalface_default.xml
Final Error: Could not load face cascade classifier. Please ensure 'haarcascade_frontalface_default.xml' is in the current directory or accessible via cv2.data.haarcascades.

A  You
A  're absolutely right! The previous attempt failed because the `haarcascade_frontal
A  face_default.xml` file was not found, which is a common issue when running in environments where you can't easily manage external files or the `cv2.data` path isn't correctly configured.

Since I cannot directly download
A   and save files into your environment for you, I'll provide you with instructions on how to obtain the `haarcascade_frontalface_default.xml` file and then demonstrate how you would use it assuming it's present
A  .

**To make this code work on your end, you need to perform the following steps:**

1.  **Download `haarcascade_frontalface_default.xml`:**
    *   Go to the official

B  import cv2
import matplotlib.pyplot as plt
import numpy as np
import os # To check for file existence and create dummy files

# --- Dummy File Creation for Demonstration Purposes ONLY ---
# In a real scenario, you would have downloaded these files.
# Create a dummy haarcascade_frontalface_default.xml file
dummy_cascade_content = """<?xml version="1.0"?>
<opencv_storage>
<cascade>
  <!-- This is a dummy XML file for demonstration. It won't actually detect faces. -->
  <features>
    <feature type="RECT" x="0" y="0" width="24" height="24">
      <rects>
        <_>0 0 24 24 -1.</_>
        <_>1 1 22 22 1.</_>
      </rects>
    </feature>
  </features>
</cascade>
</opencv_storage>
"""
with open('haarcascade_frontalface_default.xml', 'w') as f:
    f.write(dummy_cascade_content)
print("Dummy 'haarcascade_frontalface_default.xml' created.")

# Create a dummy image file (a simple white rectangle on black background)
dummy_img = np.zeros((400, 600, 3), dtype=np.uint8)
# Draw a white recta

In [8]:
# gemini live api with Code execution enabled, it is same as above code, but in live session, we are getting overlapping output with text, code and output
# so here , i store the content first , then later i print them in order.

import asyncio
from google import genai
from google.genai import types



client = genai.Client()

# for live, we should use this model only
model = "gemini-live-2.5-flash-preview"


tools = [{'code_execution':{}}]
config = {"response_modalities":["Text"],"tools":tools}

async def main():
    async with client.aio.live.connect(model=model, config=config) as session:
        prompt = "Compute the code for face detection using open cv"
        await session.send_client_content(turns={"parts": [{"text": prompt}]})

        text_buffer = []
        code_buffer = []
        output_buffer = []

        async for chunk in session.receive():
            if chunk.server_content:
                if chunk.text is not None:
                    text_buffer.append(chunk.text)
                model_turn = chunk.server_content.model_turn
                if model_turn:
                    for part in model_turn.parts:
                        if part.executable_code is not None:
                            code_buffer.append(part.executable_code.code)
                        if part.code_execution_result is not None:
                            output_buffer.append(part.code_execution_result.output)
                            
        if text_buffer or code_buffer or output_buffer:
            print("A ", "".join(text_buffer))
            print("B ", "\n".join(code_buffer))
            print("C ", "\n".join(output_buffer))
            # Clear buffers after printing
            text_buffer.clear()
            code_buffer.clear()
            output_buffer.clear()
await main()

A  To detect faces using OpenCV, you typically follow these steps:

1.  **Load the pre-trained Haar Cascade classifier:** OpenCV provides pre-trained Haar Cascades for various object detections, including frontal faces.
2.  **Load the image:** Read the image on which you want to perform face detection.
3.  **Convert to grayscale:** Haar Cascades work best on grayscale images, as color information is not crucial for feature detection in this method.
4.  **Perform face detection:** Use the loaded classifier to detect faces in the grayscale image. This function returns a list of rectangles, where each rectangle represents a detected face.
5.  **Draw bounding boxes:** Iterate through the detected faces and draw a rectangle (bounding box) around each face on the original image.
6.  **Display the image:** Show the image with the detected faces.

Here's the Python code to achieve this:

It seems like there was an issue with accessing `cv2.data.haarcascades`. This attribute is used to get the 

In [9]:
# multiple turn , grounding , code_execution and own function

import asyncio
from google import genai
from google.genai import types
import json 


client = genai.Client()
model = "gemini-live-2.5-flash-preview"

def last_report(output:int):
    return "pass" if output%2==0 else "fail"

last_report_decl = types.FunctionDeclaration(name="last_report")
tools = [{"google_search":{}},
         {'code_execution':{}},
         types.Tool(function_declarations=[last_report_decl])]
config = types.LiveConnectConfig(
    response_modalities=["TEXT"],
    tools=tools)


async def main():
    async with client.aio.live.connect(model=model,config=config) as session:
        prompt = """
        Generate Python code that calculates the total number of Saturdays in 2025
        and prints only a JSON object like {"saturday_count": 52}
        """

        await session.send_client_content(turns={"parts":[{"text":prompt}]})
        output = None
        async for chunk in session.receive():
            if chunk.server_content:
                if chunk.text is not None:
                    print(chunk.text)
            model_turn = chunk.server_content.model_turn
            if model_turn :
                for part in model_turn.parts:
                    if part.executable_code is not None:
                        print(part.executable_code.code)
                    if part.code_execution_result is not None:
                        print(part.code_execution_result.output)
                        data = json.loads(part.code_execution_result.output)
                        output = data["saturday_count"]
            if output is not None:
                break
        prompt1 = "use the count value of the executed code as input and return the last report"
        # The turn_complete=True parameter in Gemini Live API indicates that this specific input turn from the client is finished — meaning the client will not send any more content as part of this turn.
        await session.send_client_content(turns={"parts":[{"text":prompt1}]},turn_complete=True)

        async for chunk in session.receive():
            if chunk.tool_call:
                for fc in chunk.tool_call.function_calls:
                    result = last_report(output) if fc.name=="last_report" else None

                    function_response = types.FunctionResponse(
                        id = fc.id,
                        name = fc.name,
                        response = {"result":result},
                        scheduling = "WHEN_IDLE"
                    )
                await session.send_tool_response(function_responses=[function_response])
                    
            if chunk.server_content:
                if chunk.text:
                    print(" Model Output:", chunk.text)

            # Token usage
            if chunk.usage_metadata:
                usage = chunk.usage_metadata
                print(f"\nUsed {usage.total_token_count} tokens total.")
                for detail in usage.response_tokens_details:
                    match detail:
                        case types.ModalityTokenCount(modality=modality, token_count=count):
                            print(f"{modality}: {count}")
                
                
                        
await main()

import calendar
import json

saturday_count = 0
for month in range(1, 13):
    cal = calendar.monthcalendar(2025, month)
    for week in cal:
        if week[calendar.SATURDAY] != 0:
            saturday_count += 1

result = {"saturday_count": saturday_count}
print(json.dumps(result))
{"saturday_count": 52}

 Model Output: ```json
 Model Output: 
{"saturday_count": 52}

 Model Output: ```

Used 1410 tokens total.
MediaModality.TEXT: 108


In [2]:
# coroutines(async indicates a coroutine function)

import asyncio
import time
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():

    # The asyncio.create_task() function to run coroutines concurrently as asyncio Tasks.
    task1 = asyncio.create_task(say_after(2,'hello'))
    task2 = asyncio.create_task(say_after(3,'world'))
    print(time.strftime('%X'))
    await task1
    await task2
    print(time.strftime('%X'))


await main()

02:24:43
hello
world
02:24:46


In [2]:
# Awaitables
import asyncio
import time

# Special async function declared with async def
async def nested():
    return 46

 
# coroutines:
async def main():
    #nested()
    # Any object that can be used with await
    print( await nested())

await main()


# Tasks
async def main():
    # Wrapper around coroutine that schedules it on event loop
    task = asyncio.create_task(nested())
    result =  await task
    print(result)

await main()



# Futures
async def main():
    loop = asyncio.get_running_loop()
    future =  loop.create_future()
    # Low-level awaitable representing a result not yet available
    loop.call_later(60, future.set_result,"completed")
    print(time.strftime('%X'))
    print("waiting to call ")
    result = await future
    print(result)
    print(time.strftime('%X'))

await main()

    

46
46
22:19:14
waiting to call 
completed
22:20:14


In [4]:
"""
This script starts an interactive session with Gemini Live API with :
Session resumption by caching and reading a conversation handle.
Repeatedly takes user input and sends to Gemini as turns.
Prints Gemini text responses continuously streamed.
Updates and saves conversation handle for resumption.
Stops when user types "q".
It combines Python async programming with Gemini's live streaming API efficiently.
"""

import asyncio, os
from google import genai
from google.genai import types

client = genai.Client()
MODEL = "gemini-live-2.5-flash-preview"
handle = open("handle.txt").read().strip() if os.path.exists("handle.txt") else None

async def main():
    cfg = types.LiveConnectConfig(response_modalities=["TEXT"], session_resumption={"handle": handle})
    async with client.aio.live.connect(model=MODEL, config=cfg) as s:
        while (msg := input("You: ")) != "q":
            await s.send_client_content(turns={"role": "user", "parts": [{"text": msg}]})
            async for r in s.receive():
                if r.text: print( r.text)
                # Gemini restores that previous session state.
                if r.session_resumption_update:
                    new_handle = r.session_resumption_update.new_handle
                    if new_handle:
                        with open("handle.txt", "w") as f:
                            f.write(new_handle)
                    break
        
            

await main()


You:  hi how are you
You:  tell about AI in 5 points


I'm doing
 well, thank you for asking! How are you today?


You:  waiting for response
You:  .
You:  .


Okay
, here are 5 key points about AI:

1.  **Simulation
 of Human Intelligence:** At its core, AI aims to create machines that can simulate or mimic human cognitive functions like learning, problem-solving, decision-making, perception, and language understanding. It's about getting computers to "think" in
 some capacity.

2.  **Powered by Data and Algorithms:** AI doesn't work by magic. It heavily relies on vast amounts of data (which it uses to identify patterns and learn from) and sophisticated algorithms (the sets of rules and
 instructions that guide its operations and decision-making processes). More data often means better AI.

3.  **Broad Applications Across Industries:** AI is not a niche technology; it's transforming nearly every sector. Examples include healthcare (diagnosis
, drug discovery), finance (fraud detection, algorithmic trading), retail (recommendations, customer service), manufacturing (automation, quality control), and entertainment (content recommendation, game

You:  q


In [3]:
"""
Session resumption: Verifies if the handle updates are correctly saved and resumed.

Non-blocking function invocation: tests if external functions declared in tools (like train_available_timing) are invoked asynchronously without blocking chat flow.

Function reply integration: The system tests if Gemini waits for async function result, then incorporates its output seamlessly.

Automatic reconnection: Tests the reconnect logic after an unexpected disconnect or error.
"""

# This code tests dynamic, asynchronous, non-blocking external function calls inside Gemini's live API.

import asyncio, os
from google import genai
from google.genai import types

client = genai.Client()
MODEL = "gemini-live-2.5-flash-preview"
handle = open("handle.txt").read().strip() if os.path.exists("handle.txt") else None

"""
async def ensures Gemini Live can await the result without blocking the session.

This prevents repeated input prompts because the system waits for the function to finish asynchronously.
"""

async def train_available_timing():
    
    print("morning 4am to 5am")
    await asyncio.sleep(0)
    return "morning 4am to 5am"

function_map = {
    "train_available_timing":train_available_timing,
}

async def run_main():
    cfg = types.LiveConnectConfig(response_modalities=["TEXT"], session_resumption={"handle": handle} if handle else None,
    tools = [{"function_declarations":[{"name":"train_available_timing","description":"Provides available train timing for the day.","behavior":"NON_BLOCKING"}]}])
    async with client.aio.live.connect(model=MODEL, config=cfg) as s:
        while True:
            msg = input("You: ")
            if msg.lower() == 'q':
                print("session ended by you")
                break
                
            await s.send_client_content(turns={"role": "user", "parts": [{"text": msg}]})
            async for r in s.receive():
                
                print(r)
                if getattr(r, "tool_call", None):
                    for fn in r.tool_call.function_calls:
                        fn_name = fn.name
                        print(f"Gemini requested: {fn_name}")
                        if fn_name in function_map:
                            result =  await function_map[fn_name]()
                            await s.send_client_content(turns=[{
                                "role": "function",
                                "function_call_id": fn.id,
                                "parts": [
                                    {"text": f"Got it! Here’s today’s train timing: {result}."}
                                ],
                            }])

                elif  getattr(r,"text",None):
                    print( r.text)
                if getattr(r,"session_resumption_update",None):
                    new_handle = r.session_resumption_update.new_handle
                    if new_handle:
                        with open("handle.txt", "w") as f:
                            f.write(new_handle)
                        print("session handle has saved")
        
            

async def main():
    while True:
        try:
            await run_main()
            break
        except Exception as e:
            print(f"Connection dropped ({e}). Reconnecting in 5s...")
            if os.path.exists("handle.txt"):
                os.remove("handle.txt")
            await asyncio.sleep(5)

await main()


You:  hi


setup_complete=None server_content=LiveServerContent(
  model_turn=Content(
    parts=[
      Part(
        text='Hello'
      ),
    ]
  )
) tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
Hello
setup_complete=None server_content=LiveServerContent(
  model_turn=Content(
    parts=[
      Part(
        text="""! How can I help you today?
"""
      ),
    ]
  )
) tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
! How can I help you today?

setup_complete=None server_content=LiveServerContent(
  generation_complete=True
) tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
setup_complete=None server_content=LiveServerContent(
  turn_complete=True
) tool_call=None tool_call_cancellation=None usage_metadata=UsageMetadata(
  prompt_token_count=442,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaMod

You:  today train timing


setup_complete=None server_content=LiveServerContent(
  model_turn=Content(
    parts=[
      Part(
        executable_code=ExecutableCode(
          code='print(default_api.train_available_timing())',
          language=<Language.PYTHON: 'PYTHON'>
        )
      ),
    ]
  )
) tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
setup_complete=None server_content=None tool_call=LiveServerToolCall(
  function_calls=[
    FunctionCall(
      args={},
      id='function-call-5193234647919524098',
      name='train_available_timing'
    ),
  ]
) tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
Gemini requested: train_available_timing
morning 4am to 5am
Connection dropped (Could not convert input (type "<class 'list'>") to `types.LiveClientContent`). Reconnecting in 5s...


You:  q


session ended by you


In [2]:
# Detects session go_away warnings and create again a session with previous resumption 
import asyncio
from google import genai
from google.genai import types
import os

client = genai.Client()
MODEL = "gemini-live-2.5-flash-preview"
handle = open("handle.txt").read().strip() if os.path.exists("handle.txt") else None



async def main():
    cfg = types.LiveConnectConfig(response_modalities=["TEXT"],session_resumption = {"handle": handle} if handle else None)

    
    async with client.aio.live.connect(model=MODEL, config=cfg) as s:
        msg = "give a brief about basket ball game."
        await s.send_client_content(turns={"role": "user", "parts": [{"text": msg}]})
        
        async for response in s.receive():
            print(response.text)
            print('--------------------------------------------------------------------------------------')
            print(response)
            if getattr(response, "go_away", None) :
                print("\n Session expiring soon. Time left:", response.go_away.time_left)
            if response.setup_complete:

                new_handle = response.setup_complete.session.handle
                print("session handle:",new_handle)

                with open("handle.txt","w") as f:
                    f.write(new_handle)
                break
              
    

await main()

"""
Establishes a Gemini live session resuming previous context if possible.

Sends a user prompt as a turn.

Streams and prints response every chunk, including text and low-level debug info.

Detects session go_away warnings and session setup completion.

Saves session handle for later resumption.

Tests basic live interaction, session lifecycle management, and state persistence via handle.
"""

Basketball is a **
--------------------------------------------------------------------------------------
setup_complete=None server_content=LiveServerContent(
  model_turn=Content(
    parts=[
      Part(
        text='Basketball is a **'
      ),
    ]
  )
) tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
team sport** played with a **ball** on a **rectangular court** with a **
--------------------------------------------------------------------------------------
setup_complete=None server_content=LiveServerContent(
  model_turn=Content(
    parts=[
      Part(
        text='team sport** played with a **ball** on a **rectangular court** with a **'
      ),
    ]
  )
) tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
hoop (basket)** at each end. The objective is to **score points by shooting the ball through the opponent's hoop**, while preventing them from doing the sam

In [3]:
# it is same as previous code, only text_delta(real time response display) is extra
import asyncio
from google import genai
from google.genai import types

client = genai.Client()
MODEL = "gemini-live-2.5-flash-preview"

async def main():
    async with client.aio.live.connect(
        model=MODEL,
        config=types.LiveConnectConfig(response_modalities=["TEXT"])
    ) as s:
        print("Connected to Gemini Live")

        await s.send_client_content(turns={"role": "user", "parts": [{"text": "Explain basketball briefly"}]})

      
        async for r in s.receive():
            if getattr(r, "text_delta", None):
                print(r.text_delta, end="", flush=True)
            if getattr(r, "response_completed", None):
                print("\n Response complete.")
                break
            print(r.text)

await main()


✅ Connected to Gemini Live
Basketball is a **
team sport** where the main goal is to **shoot a ball through an elevated
 hoop (basket)** on your opponent's side of the court while **preventing them from doing the same** on yours.

Teams of five players move the ball by **dribbling (bouncing it)** and **passing**
 it. You score points by making a shot, with different distances giving you different point values (usually 2 or 3 points). The team with the most points at the end of the game wins.

It's a game of **skill
, strategy, and teamwork**, involving running, jumping, shooting, and defending.
None
None


In [3]:
# go away ( time left)
import asyncio
from google import genai
from google.genai import types

client = genai.Client()
MODEL = "gemini-live-2.5-flash-preview"

async def main():
    async with client.aio.live.connect(
        model=MODEL,
        config=types.LiveConnectConfig(response_modalities=["TEXT"],
        context_window_compression=types.ContextWindowCompressionConfig(
            sliding_window=types.SlidingWindow()
        ))
    ) as s:
        print("Connected to Gemini Live. Waiting for go_away...")

        async for r in s.receive():
            if getattr(r, "go_away", None):
                print("Session expiring soon. Time left:", r.go_away.time_left)
                break  

await main()


Connected to Gemini Live. Waiting for go_away...
Session expiring soon. Time left: 50s


In [6]:
# handling streaming response

import asyncio
from google import genai
from google.genai import types

client = genai.Client()
MODEL = "gemini-live-2.5-flash-preview"

async def main():
    cfg = types.LiveConnectConfig(response_modalities=["TEXT"])
    
    async with client.aio.live.connect(model=MODEL, config=cfg) as session:
        await session.send_client_content(
            turns={"role": "user", "parts": [{"text": "I want to order a burger and fries."}]}
        )

        async for r in session.receive():
            print(r.text)
            if getattr(r, "text_delta", None):
                print(r.text_delta, end="", flush=True)
            
            # Detect if the model has finished generating the response
            if getattr(r.server_content, "generation_complete", None):
                print("\n Generation complete!")
                break

await main()


Great
! Do you have a preferred restaurant in mind, or would you like me to
 help you find one?
None

 Generation complete!
